# Updates

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
!pip install -U transformers -q
!pip install -U accelerate -q
!pip install torcharrow -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.8.0 requires pandas>=1.5.0, but you have pandas 1.3.5 which is incompatible.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.21.4 which is incompatible.
cudf-cu12 24.4.1 requires numpy<2.0a0,>=1.23

After updating packages, restart the session and then start your notebook from imports.

# Imports

In [ ]:
import torch
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torcharrow as ta
from collections import Counter
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model

In [ ]:
MODEL_ARGS = {
    'Name': 'microsoft/Phi-3-mini-128k-instruct',
    'DType': torch.bfloat16
}

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def load_model(model_args):
    model = AutoModelForCausalLM.from_pretrained(
        model_args['Name'],
        trust_remote_code=True,
        torch_dtype=model_args['DType'],
        low_cpu_mem_usage=True,
        device_map={"": device},
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args['Name'],
        trust_remote_code=True,
    )

    return model, tokenizer

In [ ]:
model, tokenizer = load_model(MODEL_ARGS)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.38k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Reading the Data

In [ ]:
def read_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

def process_data(data):
    reviews = []
    labels = []
    embeddings = []

    for item in data:
        reviews.append(item["text"])
        labels.append(item["label"])
        embeddings.append(item['embedding'])

    return reviews, labels, embeddings

def create_ta_dataframe(reviews, labels, embeddings):
    df = ta.dataframe({
        'review': reviews,
        'label': labels,
        'embedding': embeddings
    })
    return df

In [ ]:
def read_jsonlU(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

def process_data(data):
    reviews = []
    embeddings = []

    for item in data:
        reviews.append(item["text"])
        embeddings.append(item['embedding'])

    return reviews, embeddings

def create_ta_dataframe(reviews, embeddings):
    df = ta.dataframe({
        'review': reviews,
        'embedding': embeddings
    })
    return df

In [ ]:
unlabled_train_df = create_ta_dataframe(
    *process_data(
        read_jsonlU("/content/aug_imdb_unlabeled.jsonl")
    )
)

In [ ]:
FinalTest_train_df = create_ta_dataframe(
    *process_data(
        read_jsonlT("/content/test_imdb.jsonl")
    )
)

In [ ]:
train_df.describe()

  index  metric         label
-------  --------  ----------
      0  count     150
      1  mean        0.553333
      2  std         0.498813
      3  min         0
      4  25%         0
      5  50%         1
      6  75%         1
      7  max         1
dtype: Struct([Field('metric', string), Field('label', float32)]), count: 8, null_count: 0

In [ ]:
def generate_text(model, tokenizer, prompt, max_new_tokens = 100, do_sample=True, temperature=0.5):

    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    if do_sample:
        output_ids = model.generate(input_ids, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature)
    else:
        output_ids = model.generate(input_ids, max_new_tokens=max_new_tokens, do_sample=do_sample)

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return output_text[len(prompt):]

In [ ]:
prompt = """Insturction: Tell me a funny story about a cute cat
Answer:"""

generate_text(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_new_tokens=200,
    temperature=1.2,
)

' How do you say "fluffy" in cat language? Meowtiful! There once was a cat named Nibbles who lived in a big farm. She loved to chase butterflies and sunbeams, and she had the softest fur in the land. (A joke.)'

In [ ]:
examples = [
    {"text": "what an appalling piece of rubbish!!! who are all these people who blubber on about how good this is? yes, it's \"arty\"; and yes, it's \"foreign\", but .... that's not enough. the plot is boring and disjointed, like a reality show but not so slickly made.<br /><br />the people are intrinsically uninteresting; but as characters they don't have enough depth to feel empathy for them. if they are based on real people then i feel very, very sorry for them.<br /><br />the violence (and some of it is very violent) seems quite ostentatious and gratuitous. it's like the producer has visions of being quenton tarantino. not that i think very much of him, either.<br /><br />and oh yes: if i had neighbours like these, i'd move!", "label": "negative"},
    {"text": "this is exactly the sort of saturday matinee serial i loved during world war ii. i was under ten years of age. and that's the audience this serial is designed for. looking at it now, one must roar at its ineptitude and stupidity. the budget must have been next to nothing, given the shortcuts and repeats. the acting? well, this is republic pictures, 1944. they read the lines....and no doubt had one take to make them convincing.<br /><br />one and half stars. ", "label": "negative"},
    {"text": "this is the best sci-fi that i have seen in my 29 years of watching sci-fi. i also believe that dark angel will become a cult favorite. the action is great but jessica alba is the best and most gorgeous star on tv today.", "label": "positive"},
    {"text": "i give this movie 7 out of 10 because the villains were interesting in their roles and the unknown batwoman creates an interesting \"guess who\" game. the movie, however, needs more robin in it. he appeared in the movie in the beginning and sporadically throughout the rest. i always thought the new animated series did little justice to the neat new robin character, let alone knightwing. this movie just continues that bad tradition. the movie spends too much time on bruce wayne and his romance which wouldn't be so bad in one movie if the romance wasn't so unbelievable. it is still a good movie if you are a batman fan and i would recommend watching it.", "label": "positive"}
]

# Construct the prompt
def create_prompt(review_text, examples):
    prompt = "You are a sentiment analysis model. Classify the sentiment of the following IMDb movie reviews as 'positive' or 'negative'. Here are some examples:\n\n"
    for example in examples:
        prompt += f"Review: {example['text']}\nSentiment: {example['label']}\n\n"
    prompt += f"Review: {review_text}\nSentiment:"
    return prompt

# Example review to classify
review_text = "The movie was a great experience, with stunning visuals and a gripping story."

# Create the prompt
prompt = create_prompt(review_text, examples)

generate_text(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_new_tokens=200,
    temperature=1.2,
)

NameError: name 'model' is not defined